<a href="https://colab.research.google.com/github/Ahmedsayed31/Text-Summarization-T5/blob/main/Text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"


from transformers import AutoTokenizer , AutoModelForSeq2SeqLM , Trainer , TrainingArguments
from datasets import load_dataset
import torch



In [47]:
# Load dataset
dataset = load_dataset("cnn_dailymail",'3.0.0')

In [48]:
# print sample length for article and highlight
print("Sample article length:",len(dataset['train']['article'][0]),
      '\n length of highlight:',len(dataset['train']['highlights'][0]) )


Sample article length: 2527 
 length of highlight: 217


## Tokenization

In [49]:
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [50]:
# Tokenize for Summarization
def tokenize_function(data):
  input = ["summarize: "+ i for i in data['article']]
  model_inputs = tokenizer(
      input , max_length=512 ,truncation=True,padding='max_length',
      )

  # Tokenize target
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(
         data['highlights'], max_length=150 ,truncation=True,padding='max_length',
         )

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [51]:
# Apply function on dataset
tokenized_data = dataset.map(tokenize_function,batched=True)

tokenized_data.set_format('torch')

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [52]:
# split the data
train_dataset = tokenized_data['train']
test_dataset = tokenized_data['test']

# T5-Small Model

T5 (Text-to-Text Transfer Transformer) is a versatile Transformer-based model by Google that frames all NLP tasks as text-to-text problems, enabling unified pretraining and fine-tuning across diverse language tasks.

In [53]:
# Load model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [58]:
train_args= TrainingArguments(
    output_dir = "./results",
    eval_strategy='epoch',
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_steps=10,
    logging_dir='./logs'

)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [59]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer

                  )

/tmp/ipython-input-1964301010.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


## Evalution the model

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
# Define a sample text for summarization

text = """The United Nations announced a new global initiative aimed at reducing plastic waste in oceans by 50% over the next decade.
Countries will collaborate to enforce stricter regulations on plastic production and waste management systems."""


input = tokenizer("summarize: "+text , max_length=512,return_tensors='pt',truncation=True)
output = model.generate(input["input_ids"],max_length=150,num_beem=4)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n🧾 Summary:\n", summary)